## Import

In [1]:
import pandas as pd

## Introduction
In this section, the goal is to construct a high-dimensional player profile. The aim is to identify the most meaningful and promising attributes that best describe the player's characteristics.

When dealing with ratios, such as pass accuracy, the objective is to convert them to absolute numbers to capture their magnitude. Ratios are generally not retained as they may introduce redundancy.

### Defensive Action Data

In [3]:
# load data set



HelloWorld
